In [1]:
import xarray as xr 
import pandas as pd
import numpy as np
import os
import geocat.comp as gcomp


## Regrid Vertical

In [5]:
dir_in = '/glade/derecho/scratch/wchapman/ADF/FHIST_f09_g16/ts/'
filt = '/FHIST_f09_g16.cam.h0.T.197901-200512.nc'
filu = '/FHIST_f09_g16.cam.h0.Z3.197901-200512.nc'
filz = '/FHIST_f09_g16.cam.h0.Z3.197901-200512.nc'

DSt = xr.open_dataset(dir_in+filt,chunks={'time': 10}).sel(lev=1000,method='nearest')
DSz = xr.open_dataset(dir_in+filz,chunks={'time': 10})

var_name_u = 'Z3'
var_name_tbot = 'T'
var_name_z = 'Z3'

In [6]:
def process_chunk(start_idx, end_idx, DS_u, DS_tbot, DS_z, var_name_u, var_name_tbot, var_name_z, dolevs):
    dd = DS_u[var_name_u].isel(time=slice(start_idx, end_idx))
    ddps = DS_u['PS'].isel(time=slice(start_idx, end_idx))
    ddz = DS_z[var_name_z].sel(lev=1000, method='nearest').isel(time=slice(start_idx, end_idx)) * 9.81
    ddtbot = DS_tbot[var_name_tbot].isel(time=slice(start_idx, end_idx))
    hyam = DS_u['hyam']
    hybm = DS_u['hybm']

    INTER = gcomp.interpolation.interp_hybrid_to_pressure(data=dd, ps=ddps, hyam=hyam, hybm=hybm, new_levels=dolevs*100, extrapolate=True, variable='other', t_bot=ddtbot, phi_sfc=ddz)
    INTER.load()

    return INTER

In [7]:
DS_u = DSz

DS_tbot = DSt

DS_z = DSz


lendo = len(DS_u['time'])
chunk_size = 50

dolevs = np.array([80., 100., 150., 200., 250., 300., 400., 500., 600., 700., 850., 925., 1000.])
all_chunks = []

for start_idx in range(0, lendo, chunk_size):
    end_idx = min(start_idx + chunk_size, lendo)
    print(f'Processing chunk: {start_idx} to {end_idx}')
    chunk = process_chunk(start_idx, end_idx, DS_u, DS_tbot, DS_z, var_name_u, var_name_tbot, var_name_z, dolevs)
    all_chunks.append(chunk)

INTER = xr.concat(all_chunks, dim='time')

INTER = INTER.rename({'plev':'level'})
INTER['level'] = dolevs
INTER.level.attrs['long_name'] = 'pressure_level'
INTER.level.attrs['units'] = 'millibars'
INTER.level.attrs['axis'] = 'Z'

INTER = INTER.to_dataset(name=var_name_u)
INTER['time'] = INTER['time'].astype("datetime64[ns]")
INTER[var_name_u] = INTER[var_name_u].astype("float32")

splitter = f'.{var_name_u}.'
outfil = f'{dir_in}/{filu.split(splitter)[0]}.plev.{var_name_u}.{filu.split(splitter)[1]}'
print(f'Writing {outfil}')
INTER = INTER.rename({'level':'lev'})
INTER.to_netcdf(outfil)

Processing chunk: 0 to 50


/glade/u/apps/opt/conda/envs/npl-2023b/lib/python3.10/site-packages/geocat/comp/interpolation.py:133: UserWarning: Interpolation point out of data bounds encountered
  return func_interpolate(new_levels, xcoords, data, axis=interp_axis)


Processing chunk: 50 to 100


/glade/u/apps/opt/conda/envs/npl-2023b/lib/python3.10/site-packages/geocat/comp/interpolation.py:133: UserWarning: Interpolation point out of data bounds encountered
  return func_interpolate(new_levels, xcoords, data, axis=interp_axis)


Processing chunk: 100 to 150


...

Processing chunk: 250 to 300


/glade/u/apps/opt/conda/envs/npl-2023b/lib/python3.10/site-packages/geocat/comp/interpolation.py:133: UserWarning: Interpolation point out of data bounds encountered
  return func_interpolate(new_levels, xcoords, data, axis=interp_axis)


Processing chunk: 300 to 324


/glade/u/apps/opt/conda/envs/npl-2023b/lib/python3.10/site-packages/geocat/comp/interpolation.py:133: UserWarning: Interpolation point out of data bounds encountered
  return func_interpolate(new_levels, xcoords, data, axis=interp_axis)


Writing /glade/derecho/scratch/wchapman/ADF/FHIST_f09_g16/ts///FHIST_f09_g16.cam.h0.plev.Z3.197901-200512.nc
